# Потоки

#### Что такое Thread (Поток)?
Thread – это отдельный поток выполнения. Это означает, что в вашей программе могут работать две и более подпрограммы одновременно.

Для создания потоков мы будем использовать стандартный модуль threading.

In [ ]:
import threading

#### Пример многопоточной программы
Пусть два потока параллельно выводят каждый в свой файл заданное число строк. Для начала нам понадобится функция, которая выполнит задуманный нами сценарий. Аргументами целевой функции будут число строк и имя текстового файла для записи.

In [ ]:
def prescript(thefile, num):
    with open(thefile, 'w') as f:
        for i in range(num):
            if num > 500:
                f.write('ABC\n')
            else:
                f.write('abc\n')
thread1 = threading.Thread(target=prescript, args=('f1.txt', 200,))
thread2 = threading.Thread(target=prescript, args=('f2.txt', 1000))
thread1.start()
thread2.start()
thread1.join()
thread2.join()

Метод **start()** запускает ранее созданный поток. Метод **join()** останавливает поток, когда тот выполнит свои задачи и освобождает ресурсы.

##### Thread(group, target, name, args, kwargs)
Здесь **group** - группа потоков (пока не используется, должен быть равен *None*), **target** - объект, который будет вызван в методе **run()**, **name** - имя потока, **args** и **kwargs** - последовательность и словарь позиционных и именованных параметров (соответственно) для вызова заданного в параметре **target** объекта.

#### Стандартные методы работы с потоками
- current_thread()  — смотрим, какой поток вызвал функцию;
- active_count() — считаем работающие в данный момент экземпляры класса Thread;
- enumerate()   — получаем список работающих потоков.

Ещё можно управлять потоком через методы класса:
- is_alive()  —  спрашиваем поток: «Жив ещё?»  — получаем true или false;
- getName()  — узнаём имя потока;
- setName(any_name)  — даём потоку имя;

У каждого потока, пока он работает, есть уникальный идентификационный номер, который хранится в переменной **ident**.

In [ ]:
print(thread1.ident)

138760444433984


##### Timer
Отсрочить операции в вызываемых потоком функциях можно с помощью таймера. В инициализаторе объектов класса **Timer** всего два аргумента — время ожидания в секундах и функция, которую нужно в итоге выполнить:

In [ ]:
import threading
print ("Waiting...")
def timer_test():
    print ("The timer has done its job!")
tim = threading.Timer(5.0, timer_test)
tim.start()

Waiting...


In [ ]:
import threading
import time  # Для отладки

print("Waiting...")

def timer_test():
    print("The timer has done its job!")

tim = threading.Timer(5.0, timer_test)
tim.start()

# Дополнительная отладочная информация
print(f"Таймер запущен. ID потока: {tim.ident}")
print(f"Активные потоки: {threading.active_count()}")

tim.join()  # Ждём завершения
print("Главный поток завершён.")

Waiting...
Таймер запущен. ID потока: 138760444433984
Активные потоки: 5
The timer has done its job!
The timer has done its job!
Главный поток завершён.


#### Простая блокировка в Python
Взаимоисключение (mutual exception, кратко - mutex) - простейшая блокировка, которая на время работы потока с ресурсом закрывает последний от других обращений. Реализуют это с помощью класса **Lock**.

In [ ]:
import threading
mutex = threading.Lock()

Мы создали блокировку с именем mutex. Теперь её можно ставить и снимать методами **acquire()** и **release()**:

In [ ]:
resource = 0

def thread_safe_function():
    global resource
    for i in range(1000000):
        mutex.acquire()
        # Делаем что-то с переменной resource
        mutex.release()

Обратите внимание: обойти простую блокировку не может даже поток, который её активировал. Он будет заблокирован, если попытается повторно захватить ресурс, который удерживает.

In [ ]:
from threading import Thread
from time import sleep
counter = 0

def increase(by):
    global counter
    local_counter = counter
    local_counter += by
    sleep(0.1)
    counter = local_counter
    print(f'{counter=}')

t1 = Thread(target=increase, args=(10,))
t2 = Thread(target=increase, args=(20,))
t1.start()
t2.start()
t1.join()
t2.join()

counter=10
counter=20


In [ ]:
from threading import Thread, Lock
from time import sleep
counter = 0

def increase(by, lock: Lock):
    global counter
    lock.acquire()
    local_counter = counter
    local_counter += by
    sleep(0.1)
    counter = local_counter
    print(f'{counter=}')
    lock.release()

lock = Lock()
t1 = Thread(target=increase, args=(10, lock,))
t2 = Thread(target=increase, args=(20, lock,))
t1.start()
t2.start()
t1.join()
t2.join()

counter=10
counter=30


#### С блокировками и без. Пример–сравнение
Что происходит, когда два потока бьются за ресурсы, и как при этом сохранить целостность данных?

Возьмём простейшие операции инкремента и декремента (увеличения и уменьшения числа). В роли общих ресурсов выступят глобальные числовые переменные: назовём их protected_resource и unprotected_resource. К каждой обратятся по два потока: один будет в цикле увеличивать значение с 0 до 500, другой — уменьшать до 0. Первую переменную обработаем с блокировками, а вторую — без.

In [ ]:
import threading
from time import sleep

protected_resource = 0
unprotected_resource = 0
time_to_sleep = 0.00000001

NUM = 500
mutex = threading.Lock()

In [ ]:
# Потокобезопасный инкремент
def safe_plus():
    global protected_resource
    for i in range(NUM):
        # Ставим блокировку
        mutex.acquire()
        x = protected_resource
        x += 1
        sleep(time_to_sleep)
        protected_resource = x
        mutex.release()

# Потокобезопасный декремент
def safe_minus():
    global protected_resource
    for i in range(NUM):
        mutex.acquire()
        x = protected_resource
        x -= 1
        sleep(time_to_sleep)
        protected_resource = x
        mutex.release()

# То же, но без блокировки
def risky_plus():
    global unprotected_resource
    for i in range(NUM):
        x = unprotected_resource
        x += 1
        sleep(time_to_sleep)
        unprotected_resource = x

def risky_minus():
    global unprotected_resource
    for i in range(NUM):
        x = unprotected_resource
        x -= 1
        sleep(time_to_sleep)
        unprotected_resource = x

Создадим 4 потока, которые будут выполнять функции с блокировками и без:

In [ ]:
thread1 = threading.Thread(target = safe_plus)
thread2 = threading.Thread(target = safe_minus)
thread3 = threading.Thread(target = risky_plus)
thread4 = threading.Thread(target = risky_minus)
thread1.start()
thread2.start()
thread3.start()
thread4.start()
thread1.join()
thread2.join()
thread3.join()
thread4.join()
print ("Результат при работе с блокировкой %s" % protected_resource)
print ("Результат без блокировки %s" % unprotected_resource)

Результат при работе с блокировкой 0
Результат без блокировки -498


Запускаем код несколько раз подряд и видим, что полученное без блокировки значение меняется случайным образом. При использовании блокировки всё работает последовательно: сначала значение растёт, затем — уменьшается, и в итоге получаем 0. А потоки thread3 и thread4 работают без блокировки и наперебой обращаются к глобальной переменной. Каждый выполняет столько операций своего цикла, сколько успевает за время активности. Поэтому при каждом запуске получаем случайные числа.

##### Как избежать взаимных блокировок?
Следите, чтобы у нескольких блокировок не было шанса сработать одновременно. Иначе одна заглушка перекроет один поток, другая — другой, и может случиться взаимная блокировка — тупик (deadlock). Это ситуация, когда ни один поток не имеет права действовать и программа зависает или рушится.

Если есть «захват» мьютекса, ничто не должно помешать последующему «высвобождению». Это значит, что release() должен срабатывать, как только блокировка становится не нужна.

Пишите код так, чтобы блокировки снимались, даже если функция выбрасывает исключение и завершает работу нештатно. Подстраховаться можно с помощью конструкции try-except-finally:

## Deadlock - взаимный тупик

In [ ]:
from threading import Thread, Lock
from time import sleep

a = 5
b = 10

a_lock = Lock()
b_lock = Lock()

def function_a():
    global a
    global b

    a_lock.acquire()
    print('Функция a, a_lock = заблокирован')
    sleep(1)
    b_lock.acquire()
    print('Функция a, b_lock = заблокирован')

    sleep(1)

    a_lock.release()
    print('Функция a, a_lock = разблокирован')
    b_lock.release()
    print('Функция a, b_lock = разблокирован')


def function_b():
    global a
    global b

    b_lock.acquire()
    print('Функция b, b_lock = заблокирован')
    a_lock.acquire()
    print('Функция b, a_lock = заблокирован')

    sleep(1)

    b_lock.release()
    print('Функция b, b_lock = разблокирован')
    a_lock.release()
    print('Функция b, a_lock = разблокирован')


t1 = Thread(target=function_a)
t2 = Thread(target=function_b)

t1.start()
t2.start()

t1.join()
t2.join()

print('Готово')

Функция a, a_lock = заблокирован
Функция b, b_lock = заблокирован


KeyboardInterrupt: 

In [ ]:
import datetime
from threading import Semaphore, Thread
from time import sleep

s = Semaphore(2)

def semaphore_func(payload: int):
    s.acquire()
    now = datetime.datetime.now().strftime('%H:%M:%S')
    print(f'{now=}, {payload=}')
    sleep(2)
    s.release()


threads = [Thread(target=semaphore_func, args=(i,)) for i in range(7)]

for t in threads:
    t.start()

for t in threads:
    t.join()

now='11:21:48', payload=0
now='11:21:48', payload=1
now='11:21:50', payload=2
now='11:21:50', payload=3
now='11:21:52', payload=4
now='11:21:52', payload=5
now='11:21:54', payload=6
